In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import os
import matplotlib.pyplot as plt

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
current_dir = os.getcwd()

db_path = os.path.join(current_dir, "..", "db.sqlite3")

conn = sqlite3.connect(db_path)

query = """
SELECT *
FROM restaurant_product p
JOIN restaurant_orderitem oi ON p.id = oi.id
JOIN restaurant_order o ON oi.id = o.id
"""

df = pd.read_sql_query(query, conn)

df.to_csv("restaurant_data.csv", index=False)

print(f'Кількість рядків у датафреймі: {len(df)}')

conn.close()
df

# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
top_products = df.groupby("name")["quantity"].sum().reset_index()

top_10_products = top_products.sort_values(by='quantity', ascending=False).head(10)

plt.figure(figsize=(8, 8))
plt.pie(top_10_products["quantity"], labels=top_10_products["name"], autopct="%1.1f%%", startangle=90)
plt.title("Top 10 Most Popular Products by Quantity Sold")
plt.axis("equal")
plt.show()

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
df["item_price"] = df["price"] * df["quantity"]
top_products_by_price = df.groupby("name")["item_price"].sum().reset_index()

top_10_products_by_price = top_products_by_price.sort_values(by="item_price", ascending=False).head(10)
plt.figure(figsize=(8, 8))
plt.pie(top_10_products_by_price["item_price"], labels=top_10_products_by_price["name"], autopct="%1.1f%%", startangle=90)
plt.title("Top 10 Products by Revenue (Item Price)")
plt.axis("equal")
plt.show()

# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
df["datetime"] = pd.to_datetime(df["datetime"])
df["order_hour"] = df["datetime"].dt.hour
income_by_hour = df.groupby("order_hour")["item_price"].sum().reset_index()
plt.figure(figsize=(10, 6))
plt.bar(income_by_hour['order_hour'], income_by_hour['item_price'], color='skyblue')

plt.title("Total Restaurant Income by Order Hour", fontsize=14)
plt.xlabel("Hour of the Day", fontsize=12)
plt.ylabel("Total Income ($)", fontsize=12)

plt.xticks(range(24))
plt.show()

# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
df["order_day_of_week"] = df["datetime"].dt.dayofweek

income_by_day_of_week = df.groupby("order_day_of_week")["item_price"].sum().reset_index()

day_names = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
income_by_day_of_week["day_name"] = income_by_day_of_week["order_day_of_week"].apply(lambda x: day_names[x])

plt.figure(figsize=(10, 6))
plt.bar(income_by_day_of_week["day_name"], income_by_day_of_week["item_price"], color="skyblue")

plt.title("Total Restaurant Income by Day of the Week", fontsize=14)
plt.xlabel("Day of the Week", fontsize=12)
plt.ylabel("Total Income ($)", fontsize=12)

plt.xticks(rotation=45)
plt.show()